# Using cloud servers for GPU-based inference

Machine learning models are most often trained in the “cloud”, on powerful centralized servers with specialized resources (like GPU acceleration) for training machine learning models. These servers are also well-resources for inference, i.e. making predictions on new data.

In this experiment, we will use a cloud server equipped with GPU acceleration for fast inference in an image classification context.

This notebook assumes you already have a “lease” available for an RTX6000 GPU server on the CHI@UC testbed. Then, it will show you how to:

-   launch a server using that lease
-   attach an IP address to the server, so that you can access it over SSH
-   install some fundamental machine learning libraries on the server
-   use a pre-trained image classification model to do inference on the server
-   delete the server

## Launch a cloud server

We will start by preparing our environment in this notebook, then launching a cloud server using our pre-existing lease.

First, we load some required libraries:

In [ ]:
import chi
from chi import server
from chi import lease
import datetime
import os

We indicate that we’re going to use the CHI@UC site. We also need to specify the name of the Chameleon “project” that this experiment is part of. The project name will have the format “CHI-XXXXXX”, where the last part is a 6-digit number, and you can find it on your [user dashboard](https://chameleoncloud.org/user/dashboard/).

In the cell below, replace the project ID with your *own* project ID, then run the cell.

In [ ]:
chi.use_site("CHI@UC")
chi.set("project_name", "CHI-XXXXXX")

Next, we’ll specify the lease ID. This notebook assumes you already have a “lease” for an RTX6000 GPU server on CHI@UC. To get the ID of this lease,

-   Vist the CHI@UC [“reservations” page](chi.uc.chameleoncloud.org/project/leases/).
-   Click on the lease name.
-   On the following page, look for the value next to the word “Id” in the “Lease” section.

Fill in the lease ID inside the quotation marks in the following cell, then run the cell.

In [ ]:
lease_id ="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

Now, we are ready to launch a server!

First, we’ll specify the name for our server - we’ll include our username and the experiment name in the server name, so that it will be easy to identify our server in the CHI@UC web interface.

In [ ]:
username = os.environ.get("USER")
expname = "cloud-gpu"
# set a name for the server
server_name = f"{username}-{expname}".replace("_","-")

We will specify the disk image that we want our server to use.

In [ ]:
chi.set("image", "CC-Ubuntu20.04")

Then, we can create the server!

In [ ]:
server.create_server(
    server_name, 
    reservation_id=lease.get_node_reservation(lease_id),
    image_name=chi.get("image")
)
server_id = server.get_server_id(server_name)

The next cell waits for the server to be active.

In [ ]:
# wait until server is ready to use
server.wait_for_active(server_id)

Once the server is created, you should be able to see it and monitor its status on the [CHI@UC web interface](https://chi.uc.chameleoncloud.org/project/instances/). (If there was any problem while starting the server, you can also delete the server instance from that interface, in order to be able to try again.)

## Attach an address and access your server over SSH

Next, we will attach an address to our server, then use SSH to access its terminal.

First, we’ll attach an address:

In [ ]:
public_ip = server.associate_floating_ip(server_id)
server.wait_for_tcp(public_ip, port=22)

Now we can open a terminal in the Jupyter interface to access the server over SSH, using the SSH command that is printed by the following cell:

In [ ]:
print("ssh cc@%s" % public_ip)

## Install machine learning libraries on your server

In [ ]:
from chi import ssh

node = ssh.Remote(reserved_fip)

In [ ]:
node.run('sudo apt update')
node.run('sudo apt -y install python3-pip python3-dev')
node.run('sudo pip3 install --upgrade pip')

In [ ]:
node.run('wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb')
node.run('sudo dpkg -i cuda-keyring_1.0-1_all.deb')
node.run('sudo apt update')
node.run('sudo apt -y install linux-headers-$(uname -r)')
node.run('sudo apt-mark hold cuda-toolkit-12-config-common nvidia-driver-535') # don't let it install this cuda
node.run('sudo apt -y install nvidia-driver-520') # this driver likes CUDA 11.8

In [ ]:
try:
    node.run('sudo reboot') # reboot and wait for it to come up
except:
    pass
server.wait_for_tcp(reserved_fip, port=22)
node = ssh.Remote(reserved_fip) 

In [ ]:
node.run('sudo apt -y install cuda-11-8 cuda-runtime-11-8 cuda-drivers=520.61.05-1')
node.run('sudo apt -y install nvidia-gds-11-8') # install instructions say to do this separately!
node.run('sudo apt -y install libcudnn8=8.9.3.28-1+cuda11.8 nvidia-cuda-toolkit') # make sure the get cuda-11-8 version

In [ ]:
node.run("echo 'PATH=\"/usr/local/cuda-11.8/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin\"' | sudo tee /etc/environment")

In [ ]:
try:
    node.run('sudo reboot')
except:
    pass
server.wait_for_tcp(reserved_fip, port=22)
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new PATH
node.run('nvidia-smi')
node.run('nvcc --version')

In [ ]:
node.run('python3 -m pip install --user Cython==0.29.32')
node.run('wget https://raw.githubusercontent.com/teaching-on-testbeds/colab/main/requirements_chameleon_dl.txt -O requirements_chameleon_dl.txt')
node.run('python3 -m pip install --user -r requirements_chameleon_dl.txt --extra-index-url https://download.pytorch.org/whl/cu113 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html')

Test your installation - make sure Tensorflow can see the GPU:

In [ ]:
node.run('python3 -c \'import tensorflow as tf; print(tf.config.list_physical_devices("GPU"))\'')
# should say: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

## Transfering files to the server

Later in this notebook, we’ll run an image classification model - a model that accepts an image as input and “predicts” the name of the object in the image - on the server. To do this, we’ll need to upload some files to the server:

-   a sample image
-   Python code to load a model and make a prediction on the image

These are all contained in the `image_model` directory. We can upload them to the server using `scp`, and specify the source directory (in the Jupyter environment) and destination directory (on the server).

(The Python code will directly download a pre-trained model and the associated list of labels when we run it.)

In [ ]:
!scp -r image_model cc@{public_ip}:~/

## Use a pre-trained image classification model to do inference

Now, we can use the materials we uploaded to the server, and do inference - make a prediction - *on* the server.

In this example, we will use a machine learning model that is specifically designed for fast inference using GPU acceleration.

First, we need to install a couple of Python libraries on the server, including:

-   CUDA, a framework for fast machine learning operations on GPU.
-   `tensorflow` is a library for machine learning, and is capable of fast operation using GPU acceleration when a GPU is available. We’ll also get the `tensorflow-hub` library which enables us to retrieve pre-trained models shared by others at the [Tensorflow Hub](https://tfhub.dev/).
-   `Pillow` is used for image processing.

Run the following inside an SSH session on the server:

``` python
from chi import ssh

node = ssh.Remote(public_ip)
node.run('sudo apt update; sudo apt -y install cuda-11-8 libcudnn8=8.6.0.163-1+cuda11.8)
node.run('pip install nvidia-cudnn-cu11 tensorflow tensorflow-hub Pillow')
```

Then, we can execute the machine learning model! We will ask it to make a prediction for the following image:

In [ ]:
from IPython.display import Image
Image('image_model/parrot.jpg') 

In [ ]:
node.run('python /home/cc/image_model/model.py')

Make a note of the time it took to generate the prediction - would this inference time be acceptable for all applications? Also make a note of the model’s three best “guesses” regarding the label of the image - is the prediction accurate?

## Delete the server

Finally, we should stop and delete our server so that others can create new servers using the same lease. To delete our server, we can run the following cell:

In [ ]:
chi.server.delete_server(server_id)

Also free up the IP that you we attached to the server, now that it is no longer in use:

In [ ]:
ip_details = chi.network.get_floating_ip(public_ip)
chi.neutron().delete_floatingip(ip_details["id"])